# Import

In [4]:
!pip install epiweeks -q
!pip install torch_geometric -q

In [158]:
import jax
import jax.numpy as np
from jax import random
# from jax.nn import pad
from epiweeks import Week, Year
import pandas as pd
import os
import torch
dtype = torch.float
WEEKS_AHEAD = 4
PAD_VALUE = -999
DAYS_IN_WEEK = 7
NOISE_LEVELS_FLU = [0.15, 0.25, 0.50, 0.75]
NOISE_LEVELS_COVID = [0.5, 1.0, 1.5, 2.0]
from flax import linen as nn
from sklearn.preprocessing import StandardScaler
import pdb
import math
import pandas as pd
import os
import yaml
import seaborn as sns
import flax
from flax import struct
from flax.training import train_state
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from abc import ABC, abstractmethod
from scipy.stats import gamma
import math
# from flax import functional as F
import yaml
INITIAL_INFECTED_RATIO = 0.5
INFINITY_TIME = np.inf
USE_SPARSE = False
import random
import jax.nn as nn
import time
from copy import copy
import matplotlib.pyplot as plt
import pdb
from epiweeks import Week
import argparse
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import traceback
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
from jax.nn import *
from tqdm.notebook import tqdm

# Data Utils

In [159]:
datapath = "/kaggle/input/gradabmdata/Data/Processed/covid_data.csv"
county_datapath = f"/kaggle/input/gradabmdata/Data/Processed/county_data.csv"
datapath_flu_hhs = "/kaggle/input/gradabmdata/Data/Processed/flu_region_data.csv"
datapath_flu_state = "/kaggle/input/gradabmdata/Data/Processed/flu_state_data.csv"
population_path = "/kaggle/input/gradabmdata/Data/ABM_parameters/table_population.csv"
EW_START_DATA = "202012"
EW_START_DATA_FLU = "201740"  # for convenience

# Select signals COVID
macro_features = [
    "retail_and_recreation_percent_change_from_baseline",
    "grocery_and_pharmacy_percent_change_from_baseline",
    "parks_percent_change_from_baseline",
    "transit_stations_percent_change_from_baseline",
    "workplaces_percent_change_from_baseline",
    "residential_percent_change_from_baseline",
    "apple_mobility",
    "death_jhu_incidence",
    "positiveIncr",
]

# Select signals Flu
include_cols = [
    "symptom:Fever",
    "symptom:Low-grade fever",
    "symptom:Cough",
    "symptom:Sore throat",
    "symptom:Headache",
    "symptom:Fatigue",
    "symptom:Vomiting",
    "symptom:Diarrhea",
    "symptom:Shortness of breath",
    "symptom:Chest pain",
    "symptom:Dizziness",
    "symptom:Confusion",
    "symptom:Generalized tonic–clonic seizure",
    "symptom:Weakness",
]

states = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "DC",
    "FL",
    "GA",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
    "X",
]

counties = {
    "MA": [
        "25003",
#         "25005",
        "25009",
        "25011",
        "25013",
        "25015",
        "25021",
        "25023",
        "25027",
    ]
}

########################################################
#           helpers
########################################################

def convert_to_epiweek(x):
    return Week.fromstring(str(x))


def get_epiweeks_list(start_ew, end_ew):
    """
    Returns a list of epiweeks objects between start_ew and end_ew (inclusive).
    This is useful for iterating through these weeks.
    """
    if isinstance(start_ew, str):
        start_ew = convert_to_epiweek(start_ew)
    if isinstance(end_ew, str):
        end_ew = convert_to_epiweek(end_ew)
    iter_weeks = (
        list(Year(2017).iterweeks())
        + list(Year(2018).iterweeks())
        + list(Year(2019).iterweeks())
        + list(Year(2020).iterweeks())
        + list(Year(2021).iterweeks())
    )
    idx_start = iter_weeks.index(start_ew)
    idx_end = iter_weeks.index(end_ew)
    return iter_weeks[idx_start : idx_end + 1]


# def create_window_seqs(X, y, min_sequence_length):
#     """
#     Creates windows of fixed size with appended zeros.
#     Args:
#         X: Features
#         y: Targets, in synchrony with features (i.e. x[t] and y[t] correspond to the same time)
#         min_sequence_length: Minimum length of the sequence
#     """
#     # Convert to small sequences for training, starting with length 10
#     seqs = []
#     targets = []
#     mask_ys = []

#     # Starts at sequence_length and goes until the end
#     for idx in range(min_sequence_length, X.shape[0] + 1, 1):
#         # Sequences
#         seqs.append(np.array(X[:idx, :]))
#         # Targets
#         y_ = y[:idx]
#         mask_y = np.ones(len(y_))
#         targets.append(np.array(y_))
#         mask_ys.append(mask_y)
#     seqs = pad_sequence(seqs, batch_first=True, padding_value=0).type(dtype)
#     ys = pad_sequence(targets, batch_first=True, padding_value=PAD_VALUE).type(dtype)
#     mask_ys = pad_sequence(mask_ys, batch_first=True, padding_value=0).type(dtype)

#     return seqs, ys, mask_ys

def create_window_seqs(seq, ys, min_sequence_length):
    seqs = []
    ys_seqs = []
    mask_seqs = []

    for i in range(len(seq) - min_sequence_length + 1):
        seqs.append(seq[i:i+min_sequence_length])
        ys_seqs.append(ys[i:i+min_sequence_length])
        mask_seqs.append(jnp.ones(min_sequence_length))

    seqs = jnp.array(seqs)
    ys_seqs = jnp.array(ys_seqs)
    mask_seqs = jnp.array(mask_seqs)

    return seqs, ys_seqs, mask_seqs

########################################################
#           COVID: state/national level data
########################################################

def load_df(region, ew_start_data, ew_end_data):
    """Load and clean data."""
    df = pd.read_csv(datapath, low_memory=False)
    df = df[(df["region"] == region)]
    df["epiweek"] = df.loc[:, "epiweek"].apply(convert_to_epiweek)
    # Subset data using init parameters
    df = df[(df["epiweek"] <= ew_end_data) & (df["epiweek"] >= ew_start_data)]
    df = df.fillna(method="ffill")
    df = df.fillna(method="backfill")
    df = df.fillna(0)
    return df


def get_state_train_data(region, pred_week, ew_start_data=EW_START_DATA):
    """Get processed dataframe of data + target as array."""
    # Import data
    region = str.upper(region)
    pred_week = convert_to_epiweek(pred_week)
    ew_start_data = convert_to_epiweek(ew_start_data)
    df = load_df(region, ew_start_data, pred_week)
    # Select targets
    targets = df.loc[:, ["positiveIncr"]].values
    # Now subset based on input ew_start_data
    df = df[macro_features]
    return df, targets


def get_state_test_data(region, pred_week):
    """
    @param pred_week: Prediction week
    """
    pred_week = convert_to_epiweek(pred_week)
    # Import smoothed dataframe
    df = load_df(region, pred_week + 1, pred_week + 4)
    new_cases = df.loc[:, "positiveIncr"].values
    new_deaths = df.loc[:, "death_jhu_incidence"].values
    return new_cases, new_deaths


def get_train_targets_all_regions(pred_week):
    deaths_all_regions = {}
    for region in states:
        _, targets = get_state_train_data(region, pred_week)
        deaths_all_regions[region] = targets[:, 0]  # index 0 is inc positive cases
    return deaths_all_regions


def get_train_features_all_regions(pred_week):
    features_all_regions = {}
    for region in states:
        df, _ = get_state_train_data(region, pred_week)
        features_all_regions[region] = df.to_numpy()
    return features_all_regions

########################################################
#           COVID: county level data
# note: to obtain data, use get_features_per_county.ipynb
########################################################

def load_county_df(county, ew_start_data, ew_end_data):
    """Load and clean data"""
    df = pd.read_csv(county_datapath)
    df = df[(df["geo_value"] == int(county))]
    from datetime import datetime
    from datetime import date

    def convert_date_to_epiweek(x):
        if isinstance(x, Week):
            return x
        else:
            date = datetime.strptime(x, "%Y-%m-%d")
            return Week.fromdate(date)

    df["epiweek"] = df.loc[:, "time_value"].apply(convert_date_to_epiweek)
    # Subset data using init parameters
    df = df[(df["epiweek"] <= ew_end_data) & (df["epiweek"] >= ew_start_data)]
    df = df.fillna(0)  # There are zeros at the beginning
    return df


def get_county_train_data(
    county, pred_week, ew_start_data=EW_START_DATA, noise_level=0
):
    """Get processed dataframe of data + target as array"""
    # Import data
    pred_week = convert_to_epiweek(pred_week)
    ew_start_data = convert_to_epiweek(ew_start_data)
    df = load_county_df(county, ew_start_data, pred_week)
    # Select targets
    targets = df.loc[:, ["cases", "deaths"]].values
    if noise_level > 0:
        # noise_level is an index for your list
        noise = NOISE_LEVELS_COVID[noise_level - 1]
        std_vals = np.std(targets, axis=0) * noise
        noise_dist = np.random.normal(scale=std_vals, size=targets.shape)
        noisy_targets = targets + noise_dist
        noisy_targets = noisy_targets.astype("int32")
        targets = np.maximum(noisy_targets, 0)
    df.drop(columns=["epiweek", "geo_value", "time_value"], inplace=True)
    return df, targets


def get_county_test_data(county, pred_week):
    """
    @param pred_week: Prediction week
    """
    pred_week = convert_to_epiweek(pred_week)
    # Import smoothed dataframe
    df = load_county_df(county, pred_week, pred_week + 4)
    new_cases = df.loc[:, "cases"].values
    new_deaths = df.loc[:, "deaths"].values
    return new_cases, new_deaths

########################################################
#           FLU: regional/state/national level data
########################################################

def load_df_flu(region, ew_start_data, ew_end_data, geo):
    """Load and clean data"""
    if geo == "hhs":
        datapath = datapath_flu_hhs
    elif geo == "state":
        datapath = datapath_flu_state
    else:
        raise ValueError("geo must be hhs or state")
    df = pd.read_csv(datapath, low_memory=False)

    df = df[(df["region"] == region)]
    df["epiweek"] = df.loc[:, "epiweek"].apply(convert_to_epiweek)
    # Subset data using init parameters
    df = df[(df["epiweek"] <= ew_end_data) & (df["epiweek"] >= ew_start_data)]
    df = df.fillna(method="ffill")
    df = df.fillna(method="backfill")
    df = df.fillna(0)
    return df


def get_state_train_data_flu(
    region, pred_week, ew_start_data=EW_START_DATA_FLU, geo="state", noise_level=0
):
    """Get processed dataframe of data + target as array"""
    # Import data
    region = str.upper(region)
    pred_week = convert_to_epiweek(pred_week)
    ew_start_data = convert_to_epiweek(ew_start_data)
    df = load_df_flu(region, ew_start_data, pred_week, geo)
    # Select targets
    targets = df["ili"].astype(float).values.reshape(-1, 1)  # We need this 2d
    if noise_level > 0:
        # noise_level is an index for your list
        noise = NOISE_LEVELS_FLU[noise_level - 1]
        NOISE_STD = targets.std() * noise
        noise_dist = np.random.normal(loc=0, scale=NOISE_STD, size=targets.shape)
        noisy_targets = targets + noise_dist
        targets = np.array([max(ix, 0) for ix in noisy_targets])
    # Now subset based on input ew_start_data
    df = df[["month"] + include_cols]
    return df, targets


def get_state_test_data_flu(region, pred_week, geo="state"):
    """
    @param pred_week: Prediction week
    """
    pred_week = convert_to_epiweek(pred_week)
    # Import smoothed dataframe
    df = load_df_flu(region, pred_week + 1, pred_week + 4, geo)
    ili = df.loc[:, "ili"].values
    return ili


def get_dir_from_path_list(path):
    outdir = path[0]
    if not (os.path.exists(outdir)):
        os.makedirs(outdir)
    for p in path[1:]:
        outdir = os.path.join(outdir, p)
        if not (os.path.exists(outdir)):
            os.makedirs(outdir)
    return outdir

# Model Utils

In [160]:
SMOOTH_WINDOW = 7

class TransformerAttn(nn.Module):
    """
    Module that calculates self-attention weights using transformer-like attention
    """

    dim_in: int
    value_dim: int
    key_dim: int

    def setup(self):
        self.value_layer = nn.Dense(features=self.value_dim)
        self.query_layer = nn.Dense(features=self.value_dim)
        self.key_layer = nn.Dense(features=self.key_dim)

    def __call__(self, seq):
        seq_in = seq.transpose((1, 0, 2))
        value = self.value_layer(seq_in)
        query = self.query_layer(seq_in)
        keys = self.key_layer(seq_in)
        weights = (jnp.matmul(value, query.transpose((0, 2, 1)))) / jnp.sqrt(seq.shape[-1])
        weights = nn.softmax(weights, axis=-1)
        return jnp.matmul(weights, keys).transpose((1, 0, 2))

    def forward_mask(self, seq, mask):
        seq_in = seq.transpose((1, 0, 2))
        value = self.value_layer(seq_in)
        query = self.query_layer(seq_in)
        keys = self.key_layer(seq_in)
        weights = (jnp.matmul(value, query.transpose((0, 2, 1)))) / jnp.sqrt(seq.shape[-1])
        weights = jnp.exp(weights)
        weights = (weights.transpose((1, 2, 0)) * mask.transpose((1, 0))).transpose((1, 2, 0))
        weights = weights / (weights.sum(-1, keepdims=True))
        return jnp.matmul(weights, keys).transpose((1, 0, 2)) * mask


class EmbedAttenSeq(nn.Module):
    """
    Module to embed a sequence. Adds Attention module.
    """

    dim_seq_in: int
    dim_metadata: int
    rnn_out: int
    dim_out: int
    n_layers: int
    bidirectional: bool
    attn = TransformerAttn
    dropout: float

    def setup(self):
        self.rnn = nn.GRU(
            self.rnn_out // 2 if self.bidirectional else self.rnn_out,
            num_layers=self.n_layers,
            bidirectional=self.bidirectional,
            dropout=self.dropout,
        )
        self.attn_layer = self.attn()
        self.out_layer = nn.Sequential(
            nn.Dense(features=self.dim_out),
            nn.Tanh(),
            nn.Dropout(rate=self.dropout),
        )

    def forward_mask(self, seqs, metadata, mask):
        latent_seqs = self.rnn(seqs)[0]
        latent_seqs = latent_seqs
        latent_seqs = self.attn_layer.forward_mask(latent_seqs, mask)
        latent_seqs = jnp.sum(latent_seqs, axis=0)
        out = self.out_layer(jnp.concatenate([latent_seqs, metadata], axis=1))
        return out

    def forward(self, seqs, metadata):
        latent_seqs, encoder_hidden = self.rnn(seqs)
        latent_seqs = self.attn_layer(latent_seqs).sum(0)
        out = self.out_layer(jnp.concatenate([latent_seqs, metadata], axis=1))
        return out, encoder_hidden

class DecodeSeq(nn.Module):
    """
    Module to embed a sequence. Adds Attention module.
    """

    dim_seq_in: int
    dim_metadata: int
    rnn_out: int
    dim_out: int
    n_layers: int
    bidirectional: bool
    dropout: float

    def setup(self):
        self.act_fcn = nn.Tanh()
        self.embed_input = nn.Dense(features=self.rnn_out)
        self.attn_combine = nn.Dense(features=self.rnn_out)
        self.rnn = nn.GRU(
            self.rnn_out // 2 if self.bidirectional else self.rnn_out,
            num_layers=self.n_layers,
            bidirectional=self.bidirectional,
            dropout=self.dropout,
        )
        self.out_layer = nn.Sequential(
            nn.Dense(features=self.dim_out),
            nn.Tanh(),
            nn.Dropout(rate=self.dropout),
        )

    def forward(self, Hi_data, encoder_hidden, context):
        inputs = Hi_data.transpose((1, 0, 2))
        if self.bidirectional:
            h0 = encoder_hidden[2:].transpose((1, 0, 2))
        else:
            h0 = jnp.stack(encoder_hidden[2:], axis=0).sum(0).unsqueeze(0)
        inputs = self.embed_input(inputs)
        context = jnp.tile(context, (inputs.shape[0], 1, 1))
        inputs = jnp.concatenate((inputs, context), axis=2)
        inputs = self.attn_combine(inputs)
        latent_seqs, _ = self.rnn(inputs, h0)
        latent_seqs = latent_seqs.transpose((1, 0, 2))
        latent_seqs = self.out_layer(latent_seqs)
        return latent_seqs


def moving_average(x, w):
    return jnp.array(pd.Series(x).rolling(w, min_periods=1).mean().values)


def fetch_county_data_covid(
    state="MA", county_id="25005", pred_week="202021", batch_size=32, noise_level=0
):
    np.random.seed(17)

    if county_id == "all":
        all_counties = counties[state]
    else:
        all_counties = [county_id]

    c_seqs = []
    c_ys = []
    for county in all_counties:
        X_county, y = get_county_train_data(county, pred_week, noise_level=noise_level)
        y = moving_average(y[:, 1].ravel(), SMOOTH_WINDOW).reshape(-1, 1)
        c_seqs.append(X_county.to_numpy())
        c_ys.append(y)
    c_seqs = jnp.array(c_seqs)
    c_ys = jnp.array(c_ys)

    scalers = [StandardScaler() for _ in range(len(all_counties))]
    c_seqs_norm = []
    for i, scaler in enumerate(scalers):
        c_seqs_norm.append(scaler.fit_transform(c_seqs[i]))
    c_seqs_norm = jnp.array(c_seqs_norm)

    county_idx = {r: i for i, r in enumerate(all_counties)}

    def one_hot(idx, dim=len(county_idx)):
        ans = jnp.zeros(dim, dtype=jnp.float32)
        ans = ans.at[idx].set(1.0)
        return ans

    metadata = jnp.array([one_hot(county_idx[r]) for r in all_counties])

    min_sequence_length = 5
    metas, seqs, y, y_mask = [], [], [], []
    for meta, seq, ys in zip(metadata, c_seqs_norm, c_ys):
        seq, ys, ys_mask = create_window_seqs(seq, ys, min_sequence_length)
        metas.append(meta)
        seqs.append(seq[(-1,)])  # Modified indexing here
        y.append(ys[(-1,)])  # Modified indexing here
        y_mask.append(ys_mask[(-1,)])  # Modified indexing here

    all_metas = jnp.array(metas, dtype=jnp.float32)
    all_county_seqs = jnp.concatenate(seqs, axis=0)
    all_county_ys = jnp.concatenate(y, axis=0)
    all_county_y_mask = jnp.concatenate(y_mask, axis=0)

    counties_train, metas_train, X_train, y_train, y_mask_train = (
        all_counties,
        all_metas,
        all_county_seqs,
        all_county_ys,
        all_county_y_mask,
    )

    train_dataset = SeqData(counties_train, metas_train, X_train, y_train, y_mask_train)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )

    seqlen = all_county_seqs.shape[1]
    return train_loader, metas_train.shape[-1], X_train.shape[-1], seqlen



class SeqData(torch.utils.data.Dataset):
    def __init__(self, counties, metas, seqs, ys, ys_mask):
        self.counties = counties
        self.metas = metas
        self.seqs = seqs
        self.ys = ys
        self.ys_mask = ys_mask

    def __len__(self):
        return len(self.counties)

    def __getitem__(self, index):
        county = self.counties[index]
        meta = self.metas[index]
        seq = self.seqs[index]
        y = self.ys[index]
        y_mask = self.ys_mask[index]
        return county, meta, seq, y, y_mask

class ODE(nn.Module):
    def __init__(self, params, device):
        super(ODE, self).__init__()
        county_id = params["county_id"]
        abm_params = f"/kaggle/input/gradabmdata/Data/{county_id}_generated_params.yaml"
        # Reading params
        with open(abm_params, "r") as stream:
            try:
                abm_params = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print("Error in reading parameters file")
                print(exc)
        params.update(abm_params)
        self.params = params
        self.device = device
        self.num_agents = self.params["num_agents"]  # Population


class SEIRM(ODE):
    def __init__(self, params, device):
        super().__init__(params, device)

    def init_compartments(self, learnable_params):
        """let's get initial conditions"""
        initial_infections_percentage = learnable_params[
            "initial_infections_percentage"
        ]
        initial_conditions = jnp.empty((5))
        no_infected = (
            initial_infections_percentage / 100
        ) * self.num_agents  # 1.0 is ILI
        initial_conditions = initial_conditions.at[2].set(no_infected)
        initial_conditions = initial_conditions.at[0].set( self.num_agents - no_infected)
        print("initial infected", no_infected)
        self.state = initial_conditions

    def step(self, t, values):
        """
        Computes ODE states via equations
            state is the array of state value (S,E,I,R,M)
        """
        params = {
            "beta": values[0],
            "alpha": values[1],
            "gamma": values[2],
            "mu": values[3],
            "initial_infections_percentage": values[4],
        }
        if t == 0:
            self.init_compartments(params)
        # to make the NN predict lower numbers, we can make its prediction to be N-Susceptible
        dSE = params["beta"] * self.state[0] * self.state[2] / self.num_agents
        dEI = params["alpha"] * self.state[1]
        dIR = params["gamma"] * self.state[2]
        dIM = params["mu"] * self.state[2]

        dS = -1.0 * dSE
        dE = dSE - dEI
        dI = dEI - dIR - dIM
        dR = dIR
        dM = dIM

        # concat and reshape to make it rows as obs, cols as states
        self.dstate = jnp.stack([dS, dE, dI, dR, dM], 0)
        NEW_INFECTIONS_TODAY = dEI
        NEW_DEATHS_TODAY = dIM
        # update state
        self.state = self.state + self.dstate

        return NEW_INFECTIONS_TODAY, NEW_DEATHS_TODAY

class SIRS(ODE):
    def __init__(self, params, device):
        super().__init__(params, device)

    def init_compartments(self, learnable_params):
        """let's get initial conditions"""
        initial_infections_percentage = learnable_params[
            "initial_infections_percentage"
        ]
        initial_conditions = jnp.empty((2))
        no_infected = (
            initial_infections_percentage / 100
        ) * self.num_agents  # 1.0 is ILI
        initial_conditions = initial_conditions.at[1].set(no_infected)
        initial_conditions = initial_conditions.at[0].set(self.num_agents - no_infected)
        print("initial infected", no_infected)

        self.state = initial_conditions

    def step(self, t, values):
        """
        Computes ODE states via equations
            state is the array of state value (S,I)
        """
        params = {
            "beta": values[0],  # contact rate, range: 0-1
            "initial_infections_percentage": values[1],
        }
        # set from expertise
        params["D"] = 3.5
        params["L"] = 2000
        if t == 0:
            self.init_compartments(params)
        dS = (self.num_agents - self.state[0] - self.state[1]) / params["L"] - params[
            "beta"
        ] * self.state[0] * self.state[1] / self.num_agents
        dSI = params["beta"] * self.state[0] * self.state[1] / self.num_agents
        dI = dSI - self.state[1] / params["D"]

        # concat and reshape to make it rows as obs, cols as states
        self.dstate = jnp.stack([dS, dI], 0)

        NEW_INFECTIONS_TODAY = dSI
        # ILI is percentage of outpatients with influenza-like illness
        # ILI = params['lambda'] * dSI / self.num_agents
        ILI = dSI / self.num_agents * 100  # multiply 100 because it is percentage

        # update state
        self.state = self.state + self.dstate
        return NEW_INFECTIONS_TODAY, ILI


# Abm-model

In [229]:
from jax import random, lax
import flax.linen as nn

class LogitRelaxedBernoulli(nn.Module):
    temperature: float = 0.3

    def setup(self):
        self.temperature = self.param("temperature", nn.initializers.constant(self.temperature))

    def rsample(self, rng_key, shape):
        eps = random.uniform(rng_key, shape, minval=1e-6, maxval=1 - 1e-6)
        logits = self.logits
        y = (logits + jnp.log(eps) - jnp.log(1.0 - eps)) / self.temperature
        return y

    def log_prob(self, value):
        return (
            jnp.log(self.temperature)
            - self.temperature * value
            + self.logits
            - 2 * nn.softplus(-self.temperature * value + self.logits)
        )

class InfectionNetwork(MessagePassing):
    lam: nn.Module
    R: nn.Module
    SFSusceptibility: jnp.ndarray
    SFInfector: jnp.ndarray
    lam_gamma_integrals: jnp.ndarray

    def setup(self, lam,R,SFSusceptibility,SFInfector,lam_gamma_integrals):
        self.lam = lam
        self.R = R
        self.SFSusceptibility = SFSusceptibility
        self.SFInfector = SFInfector
        self.lam_gamma_integrals = lam_gamma_integrals

    def forward_sparse(self, data, r0_value_trainable):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr
        t = data.t
        # sparse adjacency matrix of inter-agent interactions
        S_A_s = self.SFSusceptibility[x[:, 0].astype(jnp.int32)]
        A_s_i = self.SFInfector[x[:, 1].astype(jnp.int32)]
        integrals = jnp.zeros_like(S_A_s)
        infected_idx = x[:, 2].astype(jnp.bool_)
        infected_times = t - x[infected_idx, 3]
        integrals = integrals.at[infected_idx].set(self.lam_gamma_integrals[infected_times.astype(jnp.int32)],)
        I_bar = x[:, 4 + 22]  # only info for random network being used in current expts
        integral_asi = A_s_i * integrals
        sparse_adj = jax.scipy.sparse.coo_matrix(
            (jnp.ones(edge_index.shape[1]), edge_index),
            shape=(x.shape[0], x.shape[0])
        ).tocsr()
        sparse_asi = jax.scipy.sparse.coo_matrix(
            (integral_asi.reshape(-1), (edge_index[0], edge_index[1])),
            shape=(x.shape[0], 1)
        ).tocsr()
        sparse_mult = sparse_adj @ sparse_asi
        dense_mult = sparse_mult.toarray().reshape(-1)

        # total infection
        infection_transmission = (
            r0_value_trainable * S_A_s * dense_mult
        ) / I_bar  # /I_bar
        return infection_transmission.reshape(1, -1)

    def forward(self, data, r0_value_trainable):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr
        t = data.t
        return self.propagate(
            edge_index,
            x=x,
            edge_attr=edge_attr,
            t=t,
            R=r0_value_trainable,
            SFSusceptibility=self.SFSusceptibility,
            SFInfector=self.SFInfector,
            lam_gamma_integrals=self.lam_gamma_integrals,
        )

    def message(
        self,
        x_i,
        x_j,
        edge_attr,
        t,
        R,
        SFSusceptibility,
        SFInfector,
        lam_gamma_integrals,
    ):
        # x_j has shape [E, in_channels]
        tmp = self.lam(
            x_i, x_j, edge_attr, t, R, SFSusceptibility, SFInfector, lam_gamma_integrals
        )  # tmp has shape [E, 2 * in_channels]
        return tmp

from abc import ABC, abstractmethod

class DiseaseProgression(ABC, nn.Module):
    """Abstract class for disease progression"""

    @abstractmethod
    def initialize_variables(self, agents_infected_time, agents_stages, agents_next_stage_times):
        """Initialize tensor variables depending on the disease"""
        pass

    @abstractmethod
    def update_next_stage_times(self, learnable_params, newly_exposed_today, current_stages, agents_next_stage_times, t):
        """Update the time for the next stage"""
        pass

    @abstractmethod
    def update_current_stage(self, newly_exposed_today, current_stages, agents_next_stage_times, t):
        """Update the current disease stage"""
        pass

class SEIRMProgression(DiseaseProgression):
    """SEIRM for COVID-19"""

    SUSCEPTIBLE_VAR = 0
    EXPOSED_VAR = 1  # exposed state
    INFECTED_VAR = 2
    RECOVERED_VAR = 3
    MORTALITY_VAR = 4

    def __init__(self, params):
        super().__init__()
        # Default times (only for initialization, later they are learned)
        self.EXPOSED_TO_INFECTED_TIME = 3
        self.INFECTED_TO_RECOVERED_TIME = 5
        self.INFINITY_TIME = params["num_steps"] + 1
        self.num_agents = params["num_agents"]

    def initialize_variables(self, agents_infected_time, agents_stages, agents_next_stage_times):
        """Initialize tensor variables depending on the disease"""
        agents_infected_time = agents_infected_time.at[jax.numpy.logical_or(agents_stages == self.EXPOSED_VAR, agents_stages == self.INFECTED_VAR)].set(-1)
        agents_next_stage_times = agents_next_stage_times.at[agents_stages == self.EXPOSED_VAR].set(self.EXPOSED_TO_INFECTED_TIME)
        agents_next_stage_times = agents_next_stage_times.at[agents_stages == self.INFECTED_VAR].set(-1 * self.EXPOSED_TO_INFECTED_TIME)
        return agents_infected_time, agents_next_stage_times

    def update_next_stage_times(self, learnable_params, newly_exposed_today, current_stages, agents_next_stage_times, t):
        """Update the time for the next stage"""
        exposed_to_infected_time = learnable_params["exposed_to_infected_time"]
        infected_to_recovered_time = learnable_params["infected_to_recovered_time"]
        new_transition_times = agents_next_stage_times.copy()
        new_transition_times = new_transition_times.at[jnp.logical_and(current_stages == self.INFECTED_VAR, agents_next_stage_times == t)].set(self.INFINITY_TIME)
        new_transition_times = new_transition_times.at[jnp.logical_and(current_stages == self.EXPOSED_VAR, agents_next_stage_times == t)].set(t + infected_to_recovered_time)
        return jnp.where(
            newly_exposed_today,
            (t + 1 + exposed_to_infected_time),
            new_transition_times
        )

    def update_current_stage(self, newly_exposed_today, current_stages, agents_next_stage_times, t):
        """Update the current disease stage"""
        transition_to_infected = jnp.where(agents_next_stage_times <= t, self.INFECTED_VAR, self.EXPOSED_VAR)
        transition_to_mortality_or_recovered = jnp.where(agents_next_stage_times <= t, self.RECOVERED_VAR, self.INFECTED_VAR)
        stage_progression = jnp.where(
            current_stages == self.SUSCEPTIBLE_VAR,
            self.SUSCEPTIBLE_VAR,
            jnp.where(
                current_stages == self.RECOVERED_VAR,
                self.RECOVERED_VAR,
                jnp.where(
                    current_stages == self.MORTALITY_VAR,
                    self.MORTALITY_VAR,
                    jnp.where(
                        current_stages == self.EXPOSED_VAR,
                        transition_to_infected,
                        transition_to_mortality_or_recovered
                    )
                )
            )
        )
        current_stages = jnp.where(
            newly_exposed_today,
            self.EXPOSED_VAR,
            stage_progression
        )
        return current_stages

    def init_stages(self, learnable_params, device):
        """Initialize the stages"""
        initial_infections_percentage = learnable_params["initial_infections_percentage"]
        prob_infected = (initial_infections_percentage / 100) * jnp.ones((self.num_agents, 1), dtype=jnp.float32)
        p = jnp.concatenate((prob_infected, 1 - prob_infected), axis=1)
        cat_logits = jnp.log(p + 1e-9)
        agents_stages = jax.nn.gumbel_softmax(cat_logits, tau=1, hard=True, axis=1)[:, 0]
        return agents_stages

class SIRSProgression(DiseaseProgression):
    """SIRS for influenza"""

    SUSCEPTIBLE_VAR = 0
    INFECTED_VAR = 1
    RECOVERED_VAR = 2

    def __init__(self, params):
        super().__init__()
        # Default times (only for initialization, later they are learned)
        self.INFECTED_TO_RECOVERED_TIME = 5
        self.RECOVERED_TO_SUSCEPTIBLE_TIME = 100
        self.INFINITY_TIME = params["num_steps"] + 1
        self.num_agents = params["num_agents"]

    def initialize_variables(self, agents_infected_time, agents_stages, agents_next_stage_times):
        """Initialize tensor variables depending on the disease"""
        agents_infected_time = agents_infected_time.at[agents_stages == self.INFECTED_VAR].set(-1)
        agents_next_stage_times = agents_next_stage_times.at[agents_stages == self.INFECTED_VAR].set(self.INFECTED_TO_RECOVERED_TIME)
        return agents_infected_time, agents_next_stage_times

    def update_initial_times(self, learnable_params, agents_stages, agents_next_stage_times):
        infected_to_recovered_time = learnable_params["infected_to_recovered_time"]
        new_transition_times = agents_next_stage_times.set[jnp.logical_and(agents_stages == self.INFECTED_VAR, agents_next_stage_times <= t)].set(
            infected_to_recovered_time
        )
        return new_transition_times

    def get_newly_exposed(self, current_stages, potentially_exposed_today):
        newly_exposed_today = jnp.logical_and(current_stages == self.SUSCEPTIBLE_VAR, potentially_exposed_today)
        return newly_exposed_today

    def update_next_stage_times(self, learnable_params, newly_exposed_today, current_stages, agents_next_stage_times, t):
        infected_to_recovered_time = learnable_params["infected_to_recovered_time"]
        recovered_to_susceptible_time = learnable_params["recovered_to_susceptible_time"]
        new_transition_times = jnp.where(
            jnp.logical_and(current_stages == self.INFECTED_VAR, agents_next_stage_times == t),
            (t + recovered_to_susceptible_time),
            agents_next_stage_times
        )
        return jnp.where(
            newly_exposed_today,
            (t + 1 + infected_to_recovered_time),
            new_transition_times
        )

    def get_target_variables(self, params, learnable_params, newly_exposed_today, current_stages, agents_next_stage_times, t):
        new_recovered_today = jnp.where(
            jnp.logical_and(current_stages == self.INFECTED_VAR, agents_next_stage_times <= t),
            self.RECOVERED_VAR,
            self.INFECTED_VAR
        )
        ILI = jnp.sum(newly_exposed_today) / params["num_agents"] * 100
        NEW_INFECTIONS_TODAY = jnp.sum(newly_exposed_today)
        return new_recovered_today, NEW_INFECTIONS_TODAY, ILI

    def update_current_stage(self, newly_exposed_today, current_stages, agents_next_stage_times, t):
        transition_to_recovered = jnp.where(agents_next_stage_times <= t, self.RECOVERED_VAR, self.INFECTED_VAR)
        transition_to_susceptible = jnp.where(agents_next_stage_times <= t, self.SUSCEPTIBLE_VAR, self.RECOVERED_VAR)
        stage_progression = jnp.where(
            newly_exposed_today,
            self.INFECTED_VAR,
            jnp.where(
                current_stages == self.SUSCEPTIBLE_VAR,
                self.SUSCEPTIBLE_VAR,
                jnp.where(
                    current_stages == self.RECOVERED_VAR,
                    self.RECOVERED_VAR,
                    jnp.where(
                        current_stages == self.INFECTED_VAR,
                        transition_to_recovered,
                        transition_to_susceptible
                    )
                )
            )
        )
        current_stages = jnp.where(
            newly_exposed_today,
            self.INFECTED_VAR,
            stage_progression
        )
        return current_stages

    def init_stages(self, learnable_params, device):
        initial_infections_percentage = learnable_params["initial_infections_percentage"]
        prob_infected = (initial_infections_percentage / 100) * jnp.ones((self.num_agents, 1), dtype=jnp.float32)
        p = jnp.concatenate((prob_infected, 1 - prob_infected), axis=1)
        cat_logits = jnp.log(p + 1e-9)
        agents_stages = jax.nn.gumbel_softmax(cat_logits, tau=1, hard=True, axis=1)[:, 0]
        return agents_stages

import os
import yaml
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax import linen as nn
from scipy.stats import gamma
from functools import partial

class GradABM:
    def __init__(self, params, device):
        county_id = params["county_id"]
        abm_params = f"/kaggle/input/gradabmdata/Data/ABM_parameters/{county_id}_generated_params.yaml"

        # Reading params
        with open(abm_params, "r") as stream:
            try:
                abm_params = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print("Error in reading parameters file")
                print(exc)
    
        params.update(abm_params)
        params["output_location"]["parent_dir"] = "/kaggle/input/gradabmdata/Data"
        self.params = params
        self.device = device
        self.num_agents = self.params["num_agents"]
        print("Num Agents: ", self.num_agents)

        # **********************************************************************************
        # Environment state variables
        # **********************************************************************************
        # **********************************************************************************
        # Static
        self.agents_ix = jnp.arange(0, self.params["num_agents"])
        infile = os.path.join(
            self.params["output_location"]["parent_dir"],
            self.params["output_location"]["agents_dir"],
            self.params["output_location"]["agents_outfile"],
        )

        agents_df = pd.read_csv(infile)
        self.agents_ages = jnp.array(agents_df["age_group"].to_numpy())

        self.num_networks = 23
        self.network_type_dict = {}
        self.network_type_dict["random"] = 22
        self.network_type_dict_inv = {}
        self.network_type_dict_inv[22] = "random"
        
        params['disease'] = "COVID"
        # select disease progression model
        if params['disease'] == "COVID":
            self.DPM = SEIRMProgression(params)
        elif params['disease'] == "Flu":
            self.DPM = SIRSProgression(params)

        # Age and Network and Occupation may need to be checked to populate this
        self.agents_mean_interactions = jnp.zeros(
            (self.params["num_agents"], self.num_networks)
        )

        mean_int_ran_df = pd.read_csv("/kaggle/input/gradabmdata/Data/Initialization/RandomNetworkParameters.csv")
        mean_int_ran_mu = jnp.array(mean_int_ran_df["mu"].values)

        child_agents = self.agents_ages <= self.params["CHILD_Upper_Index"]
        adult_agents = jnp.logical_and(
            self.agents_ages > self.params["CHILD_Upper_Index"],
            self.agents_ages <= self.params["ADULT_Upper_Index"],
        ).reshape(-1)
        elderly_agents = self.agents_ages > self.params["ADULT_Upper_Index"]
        self.agents_mean_interactions = self.agents_mean_interactions.at[jax.ops.index[child_agents, 22]].set(mean_int_ran_mu[0])
        self.agents_mean_interactions = self.agents_mean_interactions.at[jax.ops.index[adult_agents, 22]].set(mean_int_ran_mu[1]
        )
        self.agents_mean_interactions = self.agents_mean_interactions.at[jax.ops.index[elderly_agents, 22]].set( mean_int_ran_mu[2])
        self.agents_mean_interactions_split = [
            a.reshape(-1) for a in jnp.split(self.agents_mean_interactions, 1, axis=1)
        ]

        self.R = 5.18  # learnable, but the default value
        self.R = jnp.array(self.R)
        if self.params['disease'] == "COVID":
            self.SFSusceptibility = jnp.array([0.35, 0.69, 1.03, 1.03, 1.03, 1.03, 1.27, 1.52, 1.52], dtype=jnp.float32)
            self.SFInfector = jnp.array([0.0, 0.33, 0.72, 0.0, 0.0], dtype=jnp.float32)
            self.lam_gamma = {}
            self.lam_gamma["scale"] = 5.5
            self.lam_gamma["rate"] = 2.14
        elif self.params['disease'] == "Flu":
            # from CDC Table 1, median value: https://www.cdc.gov/flu/about/keyfacts.htm
            self.SFSusceptibility = jnp.array([13.2, 7.9, 7.4, 7.4, 7.4, 12.0, 12.0, 3.9, 3.9], dtype=jnp.float32)
            self.SFSusceptibility = nn.softmax(self.SFSusceptibility, axis=0) * 25
            self.SFInfector = jnp.array([0.0, 0.72, 0.0], dtype=jnp.float32)
            self.lam_gamma = {}
            self.lam_gamma["scale"] = 7.5  # mean is scale/rate, CDC says it's 3-4 days
            self.lam_gamma["rate"] = 2.14

        self.B_n = {}
        self.B_n["household"] = 2
        self.B_n["occupation"] = 1
        self.B_n["random"] = 1  # 0.25

        self.lam_gamma_integrals = self._get_lam_gamma_integrals(
            **self.lam_gamma, t=self.params["num_steps"] + 10
        )  # add 10 to make sure we cover all
        self.lam_gamma_integrals = self.lam_gamma_integrals

        self.net = InfectionNetwork(
            lam,
            self.R,
            self.SFSusceptibility,
            self.SFInfector,
            self.lam_gamma_integrals,
            self.device,
        )

        self.current_time = 0
        # **********************************************************************************
        self.all_edgelist, self.all_edgeattr = self.init_interaction_graph(
            t=0
        )  # get one initial interaction graph

    def init_interaction_graph(self, t):
        """this is Part-1 of Step"""
        
        infile = os.path.join(
            get_dir_from_path_list(
                [
                    self.params["output_location"]["parent_dir"],
                    self.params["output_location"]["networks_dir"],
                    self.params["output_location"]["random_networks_dir"],
                ]
            ),
            "{}.csv".format(t),
        )

        random_network_edgelist_forward = jnp.array(pd.read_csv(infile, header=None).to_numpy().T, dtype=jnp.int32)
        random_network_edgelist_backward = jnp.vstack(
            (
                random_network_edgelist_forward[1, :],
                random_network_edgelist_forward[0, :],
            )
        )
        random_network_edgelist = jnp.hstack(
            (random_network_edgelist_forward, random_network_edgelist_backward)
        )
        random_network_edgeattr_type = jnp.ones(random_network_edgelist.shape[1], dtype=jnp.int32) * self.network_type_dict["random"]

        random_network_edgeattr_B_n = jnp.ones(random_network_edgelist.shape[1], dtype=jnp.float32) * self.B_n["random"]
        random_network_edgeattr = jnp.vstack(
            (random_network_edgeattr_type, random_network_edgeattr_B_n)
        )

        all_edgelist = jnp.hstack((random_network_edgelist,))
        all_edgeattr = jnp.hstack((random_network_edgeattr,))

        return all_edgelist, all_edgeattr

    def get_interaction_graph(self, t):
        return self.all_edgelist, self.all_edgeattr

    def init_state_tensors(self, learnable_params):
        """Initializing message passing network (currently no trainable parameters here)"""
        # Dynamic
        # a.Testing
        # b.Quarantine
        # c.Infection and Disease
        self.current_stages = self.DPM.init_stages(learnable_params, self.device)
        self.agents_infected_index = (self.current_stages > 0).astype(jnp.float32)  # Not susceptible
        self.agents_infected_time = (
            (self.params["num_steps"] + 1) * jnp.ones_like(self.current_stages)
        ).astype(jnp.float32)  # Practically infinite as np.inf gives wrong data type

        self.agents_next_stages = -1 * jnp.ones_like(self.current_stages)
        self.agents_next_stage_times = (self.params["num_steps"] + 1) * jnp.ones_like(
            self.current_stages
        ).astype(jnp.int32)  # Practically infinite as np.inf gives wrong data type

        # update values depending on the disease
        (
            self.agents_infected_time,
            self.agents_next_stage_times,
        ) = self.DPM.initialize_variables(
            self.agents_infected_time, self.current_stages, self.agents_next_stage_times
        )

        self.agents_next_stage_times = self.agents_next_stage_times.astype(jnp.float32)
        self.agents_infected_time = self.agents_infected_time.astype(jnp.float32)

    def step(self, t, param_t):
        """Send as input: r0_value [hidden state] -> trainable parameters  and t is the time-step of simulation."""
        # construct dictionary with trainable parameters
        learnable_params = {}
        if self.params['disease'] == "COVID":
            learnable_params["r0_value"] = param_t[0]
            learnable_params["mortality_rate"] = param_t[1]
            learnable_params["initial_infections_percentage"] = param_t[2]
            learnable_params["exposed_to_infected_time"] = 3
            learnable_params["infected_to_recovered_time"] = 5
        elif self.params['disease'] == "Flu":
            learnable_params["r0_value"] = param_t[0]
            learnable_params["initial_infections_percentage"] = param_t[1]
            learnable_params["infected_to_recovered_time"] = 3.5
            learnable_params["recovered_to_susceptible_time"] = 2000
        """ change params that were set in constructor """
        if t == 0:
            self.init_state_tensors(learnable_params)
            self.agents_next_stage_times = self.DPM.update_initial_times(
                learnable_params, self.current_stages, self.agents_next_stage_times
            )

        # t = self.current_time
        """Steps: i) Get interaction graph, ii) Message Passing of Infection iii) State Evolution """

        # ******************************************************************************** #
        # Part-1. Interaction Graph - Output: EdgeList, EdgeFeatures and NodeFeatures
        all_edgelist, all_edgeattr = self.get_interaction_graph(
            t
        )  # the interaction graphs for GNN at time t
        all_nodeattr = jnp.stack(
            (
                self.agents_ages,  # 0
                self.current_stages.detach(),  # 1
                self.agents_infected_index.to(self.device),  # 2
                self.agents_infected_time.to(self.device),  # 3
                *self.agents_mean_interactions_split,  # 4 to 26
                jnp.arange(self.params["num_agents"]).to(
                    self.device
                ),  # Agent ids (27)
            )
        ).T
        agents_data = Data(
            all_nodeattr,
            edge_index=all_edgelist,
            edge_attr=all_edgeattr,
            t=t,
            agents_mean_interactions=self.agents_mean_interactions,
        )

        # ******************************************************************************** #
        # Part-2. Message Passing - Transmission Dynamics + New Infections: {GNN + Variational Inference}
        # agent steps: i) collects infection [GNN]; ii) get infected based on total infection collected [Variational Inference]
        # message passing: collecting infection from neighbors
        lam_t = self.net(agents_data, learnable_params["r0_value"])
        prob_not_infected = jnp.exp(-lam_t)
        p = jnp.hstack((1 - prob_not_infected, prob_not_infected))
        cat_logits = jnp.log(p + 1e-9)
        potentially_exposed_today = nn.gumbel_softmax(
            logits=cat_logits, tau=1, hard=True, axis=1
        )[
            :, 0
        ]  # first column is prob of infections
        newly_exposed_today = self.DPM.get_newly_exposed(
            self.current_stages, potentially_exposed_today
        )

        # ******************************************************************************** #
        # Part-3. State Evolution -> Progression Dynamics {Deterministic}
        # to do here: i) update curr_stage; ii) update next_transition_time
        # self.agents_infected_time[t].sum()
        # check 2 things: i) got infected_today -> go from S to E; ii) already infected -> update E to I; I to R or M.
        # stage_progression, new_death_recovered_today = self.deterministic_stage_transition(self.agents_stages[t,:],

        # before updating, get target variables like new deaths or ILI
        # also get recovered ones
        recovered_dead_now, target1, target2 = self.DPM.get_target_variables(
            self.params,
            learnable_params,
            newly_exposed_today,
            self.current_stages,
            self.agents_next_stage_times,
            t,
        )

        # get next stages without updating yet the current_stages
        next_stages = self.DPM.update_current_stage(
            newly_exposed_today, self.current_stages, self.agents_next_stage_times, t
        )

        # update times with current_stages
        self.agents_next_stage_times = self.DPM.update_next_stage_times(
            learnable_params,
            newly_exposed_today,
            self.current_stages,
            self.agents_next_stage_times,
            t,
        )

        # safely update current_stages
        self.current_stages = next_stages

        # update for newly exposed agents {exposed_today}
        self.agents_infected_index = self.agents_infected_index.at[newly_exposed_today.astype(jnp.bool_)].set(1)
        self.agents_infected_time = self.agents_infected_time.at[newly_exposed_today.astype(jnp.bool_)].set(t)
        self.current_time = t + 1

        # ******************************************************************************** #
        # TODO: Part-4. ILI/ILI-probability update for next step -> on newly_exposed_today and current_stages
        # self.AILI_model.updateILI(current_stages.detach(), self.agents_mean_interactions, t)

        # ******************************************************************************** #
        # Wrap it all together
        wrapped_output = [recovered_dead_now, target1, target2]
        wrapped_output = [x for x in wrapped_output if x is not None]
        wrapped_output = (
            tuple(wrapped_output) if len(wrapped_output) > 1 else wrapped_output[0]
        )

        return wrapped_output

    def forward(self, x):
        # foward will be used for training and testing purposes
        output = []
        for i in range(self.params["num_steps"]):
            # inference time
            if len(x) > 0:
                output.append(self.step(i, x[i]))
            else:
                output.append(self.step(i, []))

        return output

    def process_output(self, output):
        # process the output to return meaningful data
        return output


# Train_abm

In [255]:
BENCHMARK_TRAIN = False
NUM_EPOCHS_DIFF = 15
print("---- MAIN IMPORTS SUCCESSFUL -----")
epsilon = 1e-6

MIN_VAL_PARAMS = {
"abm-covid": [
1.0,
0.001,
0.01,
], # r0, mortality rate, initial_infections_percentage
"abm-flu": [1.05, 0.1], # r0, initial_infections_percentage
"seirm": [
0.0,
0.0,
0.0,
0.0,
0.01,
], # beta, alpha, gamma, mu, initial_infections_percentage
"sirs": [0.0, 0.1], # beta, initial_infections_percentage
}
MAX_VAL_PARAMS = {
"abm-covid": [8.0, 0.02, 1.0],
"abm-flu": [2.6, 5.0],
"seirm": [1.0, 1.0, 1.0, 1.0, 1.0],
"sirs": [1.0, 5.0],
}

DAYS_HEAD = 4 * 7 # 4 weeks ahead

pi = jnp.array([math.pi], dtype=jnp.float32)

SAVE_MODEL_PATH = "/kaggle/working/Models/"

# neural network predicting parameters of the ABM
import jax.numpy as jnp
import flax.linen as nn

class CalibNN(nn.Module):
    metas_train_dim: int
    X_train_dim: int
    device: str
    training_weeks: int
    hidden_dim: int = 32
    out_dim: int = 1
    n_layers: int = 2
    scale_output: str = "abm-covid"
    bidirectional: bool = True

    def setup(self):
        """ tune """
        hidden_dim = 64
        out_layer_dim = 32

        self.emb_model = EmbedAttenSeq(
            dim_seq_in=self.X_train_dim,
            dim_metadata=self.metas_train_dim,
            rnn_out=self.hidden_dim,
            dim_out=self.hidden_dim,
            n_layers=self.n_layers,
            dropout=0.1,
            bidirectional=self.bidirectional,
        )

        self.decoder = DecodeSeq(
            dim_seq_in=1,
            rnn_out=self.hidden_dim,  # divides by 2 if bidirectional
            dim_out=out_layer_dim,
            n_layers=1,
            dropout=0.1,
            dim_metadata=1,
            bidirectional=True,
        )

        out_layer_width = out_layer_dim
        self.out_layer = [
            nn.Dense(out_layer_width // 2),
            nn.relu,
            nn.Dense(self.out_dim),
        ]

        self.min_values = jnp.array(MIN_VAL_PARAMS[self.scale_output])
        self.max_values = jnp.array(MAX_VAL_PARAMS[self.scale_output])
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, meta):
        x_embeds, encoder_hidden = self.emb_model.call(x.transpose((1, 0)), meta)
        time_seq = jnp.arange(1, self.training_weeks + WEEKS_AHEAD + 1).repeat(x_embeds.shape[0], 1).unsqueeze(2)
        Hi_data = ((time_seq - time_seq.min()) / (time_seq.max() - time_seq.min())).to(self.device)
        emb = self.decoder.call(Hi_data, encoder_hidden, x_embeds)
        out = self.out_layer(emb)
        out = self.min_values + (self.max_values - self.min_values) * self.sigmoid(out)
        return out

    
class ParamModel(nn.Module):
    def setup(self):
        self.emb_model = EmbedAttenSeq(
            dim_seq_in=self.X_train_dim,
            dim_metadata=self.metas_train_dim,
            dim_out=self.hidden_dim,
            n_layers=self.n_layers,
            dropout=0.1,
            rnn_out=self.hidden_dim,
            bidirectional=self.bidirectional,
        )
        self.layer1 = nn.Dense(features=20)
        self.layer_bypass = nn.Dense(features=20)
        self.meanfc = nn.Dense(features=self.out_dim, use_bias=True)
        
        if self.CUSTOM_INIT:
            self.meanfc.bias = nn.initializers.ones  # Set the desired initialization value
        
    def __init__(
        self,
        metas_train_dim,
        X_train_dim,
        device,
        hidden_dim=50,
        n_layers=2,
        out_dim=1,
        scale_output="abm-covid",
        bidirectional=True,
        CUSTOM_INIT=True,
    ):
        super().__init__()
        self.device = device
        self.metas_train_dim = metas_train_dim
        self.X_train_dim = X_train_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.out_dim = out_dim
        self.scale_output = scale_output
        self.bidirectional = bidirectional
        self.CUSTOM_INIT = CUSTOM_INIT
        
        self.init_weights()
        
    def init_weights(self):
        for module in self.submodules():
            if isinstance(module, nn.Dense):
                nn.initializers.xavier_uniform()(module.weight)
                module.bias = module.bias + 0.01  # Modify bias initialization if required
        
    def forward(self, x, meta):
        x_embeds = self.emb_model(x.transpose(1, 0), meta)
        
        # use embedding for predicting: i) R0 and ii) Cases {for support counties} [FOR LATER]
        ro_feats = self.layer1(x_embeds)
        ro_feats = nn.ReLU()(ro_feats)
        out = self.meanfc(ro_feats)
        
        # bound output
        out = self.apply_scaling(out)
        
        return out
    
    def apply_scaling(self, x):
        # Apply scaling based on min_values and max_values
        x = self.min_values + (self.max_values - self.min_values) * self.sigmoid(x)
        return x
    
class LearnableParams(nn.Module):
    """doesn't use data signals"""

    def __init__(self, num_params, device, scale_output="abm-covid"):
        super().__init__()
        self.device = device
        self.learnable_params = nn.Parameter(jnp.rand(num_params, device=self.device))
        self.min_values = jnp.array(MIN_VAL_PARAMS[scale_output], device=self.device)
        self.max_values = jnp.array(MAX_VAL_PARAMS[scale_output], device=self.device)
        self.sigmoid = nn.Sigmoid()

    def forward(self):
        out = self.learnable_params
        """ bound output """
        out = self.min_values + (self.max_values - self.min_values) * self.sigmoid(out)
        return out
    
def normal(x, mu, sigma_sq):
    a = (-1 * (jnp.asarray(x) - mu).pow(2) / (2 * sigma_sq)).exp()
    b = 1 / (2 * sigma_sq * pi.expand_as(sigma_sq)).sqrt()
    return a * b

def save_model(model, file_name, disease, region, week):
    PATH = os.path.join(SAVE_MODEL_PATH, disease, region)
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    jnp.savez_compressed(PATH + "/" + file_name + " " + week + ".npz", **model.params)

def load_model(model, file_name, disease, region, week, device):
    PATH = os.path.join(SAVE_MODEL_PATH, disease, region)
    params = jnp.load(PATH + "/" + file_name + " " + week + ".npz")
    model.params = params
    return model

def param_model_forward(param_model, params, x, meta):
    # get R0 from county network
    if params["model_name"].startswith("GradABM-time-varying"):
        action_value = param_model.forward(x, meta) # time-varying
    elif params["model_name"] == "ABM-expert":
        if params['disease'] == "COVID":
            action_value = jnp.array([2.5, 0.02, 0.5]) # from CDC, for COVID -- previous I0 was 0.01
    if params['disease'] == "Flu":
        action_value = jnp.array([1.3, 1.0]) # from CDC, for COVID
        action_value = jnp.repeat(action_value, meta.shape[0], 0)
    elif "ABM-pred-correction" in params["model_name"]: # same as SEIRM-static, but get
        action_value = param_model.forward()
        if params['disease'] == "COVID":
            # NOTE: to fix, beta/gamma is for SIR, maybe not the same for SEIRM
            beta = action_value[0]
            gamma = action_value[2]
            mu = action_value[3] # mortality rate
            initial_infections_percentage = action_value[4]
            action_value = jnp.stack(
            [beta / (gamma + mu), mu, initial_infections_percentage]
            )
        elif params['disease'] == "Flu":
            beta = action_value[0]
            # D = action_value[:,1]
            D = 3.5
            initial_infections_percentage = action_value[1]
            action_value = jnp.stack([beta * D, initial_infections_percentage])
            action_value = action_value.reshape(1, -1) # make sure it's 2d
            print("R0 ABM-pred-correction", action_value)
    elif "GradABM-learnable-params" in params["model_name"]:
        action_value = param_model.forward()
        action_value = jnp.repeat(action_value, meta.shape[0], 0)
    else:
        raise ValueError("model name not valid")
    return action_value

def build_param_model(params, metas_train_dim, X_train_dim, device, CUSTOM_INIT=True):
    # get param dimension for ODE
    if params['disease'] == "COVID":
        ode_param_dim = 5
        abm_param_dim = 3
        scale_output_ode = "seirm"
        scale_output_abm = "abm-covid"
    elif params['disease'] == "Flu":
        ode_param_dim = 2
        abm_param_dim = 2
        scale_output_ode = "sirs"
        scale_output_abm = "abm-flu"
    training_weeks = params["num_steps"] / 7  # only needed for time-varying
    training_weeks = int(training_weeks)
    assert training_weeks == int(training_weeks)

    """ call constructor of param model depending on the model we want to run"""
    if params["model_name"].startswith("GradABM-time-varying"):
        param_model = CalibNN(
            metas_train_dim,
            X_train_dim,
            torch.device,
            training_weeks,
            out_dim=abm_param_dim,
            scale_output= scale_output_abm,
        )
    elif params["model_name"] == "ABM-expert":
        param_model = None
    elif "ABM-pred-correction" in params["model_name"]:
        # load the param model from ODE
        # NOTE: currently it uses only R0
        param_model = LearnableParams(ode_param_dim, device, scale_output_ode).to(
            device
        )
    elif "GradABM-learnable-params" in params["model_name"]:
        param_model = LearnableParams(abm_param_dim, device, scale_output_abm).to(
            device
        )
    else:
        raise ValueError("model name not valid")
    return param_model

def build_simulator(params, devices, counties):
    """Build simulator: ABM or ODE"""
    if "ABM" in params["model_name"]:
        if params["joint"]:
            abm = {}
            # abm devices are different from the ones for the params model
            if len(devices) > 1:
                abm_devices = devices[1:]
            else:
                abm_devices = devices
            num_counties = len(counties)
            for c in range(num_counties):
                c_params = copy(params)
                c_params["county_id"] = counties[c]
                try:
                    abm[counties[c]] = GradABM(c_params, abm_devices[c % len(abm_devices)])
                except FileNotFoundError:
                    print(f"Skipping scenario for {counties[c]}. Parameter file not found.")
                    
        else:
            if len(devices) > 1:
                abm_device = devices[1]
            else:
                abm_device = devices[0]
            abm = GradABM(params, abm_device)
    
    elif "ODE" in params["model_name"]:
        disease = params.get('disease', 'DefaultDisease')
        if disease == "COVID":
            abm = SEIRM(params, devices[0])
        elif disease == "Flu":
            abm = SIRS(params, devices[0])
        else:
            print("Invalid disease specified. Using default simulator.")
            abm = DefaultODESimulator(params, devices[0])

    return abm

def forward_simulator(params, param_values, abm, training_num_steps, counties, devices):
    """Assumes abm contains only one simulator for COVID (one county), and multiple for flu (multiple counties)"""

    if params["joint"]:
        num_counties = len(counties)
        predictions = jnp.empty((num_counties, training_num_steps)).to(devices[0])
        for time_step in range(training_num_steps):
            if "time-varying" in params["model_name"]:
                param_t = param_values[:, time_step // 7, :]
            else:
                param_t = param_values
            # go over each abm
            for c in range(num_counties):
                try:
                    model_device = abm[counties[c]].device
                    population = abm[counties[c]].num_agents
                    _, pred_t = abm[counties[c]].step(
                        time_step, param_t[c].to(model_device)
                    )
                    predictions[c, time_step] = pred_t.to(devices[0])
                except KeyError:
                    print(f"Skipping county {counties[c]} as simulator object not found.")
                    abm = jnp.array([], device='cuda')
                    model_device = 0
                    population = 0
                    _, pred_t = abm[counties[c]].step(
                        time_step, param_t[c].to(model_device)
                    )
                    predictions[c, time_step] = pred_t.to(devices[0])
                    
    else:
        num_counties = 1
        param_values = param_values.squeeze(0)
        predictions = []
        for time_step in range(training_num_steps):
            if "time-varying" in params["model_name"]:
                param_t = param_values[time_step // 7, :]
            else:
                param_t = param_values
            try:
                model_device = abm.device
                _, pred_t = abm.step(time_step, param_t.to(model_device))
                predictions.append(pred_t.to(devices[0]))
            except KeyError:
                print("Simulator object not found. Skipping simulation.")
        predictions = jnp.stack(predictions, 0).reshape(1, -1)  # num counties, seq len

    # post-process predictions for flu
    # targets are weekly, so we have to convert from daily to weekly
    if params['disease'] == "Flu":
        predictions = predictions.reshape(num_counties, -1, 7).sum(2)
    else:
        predictions = predictions.reshape(num_counties, -1)

    return predictions.unsqueeze(2)

import time
import os
import numpy as np
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
from flax.training import checkpoints
from flax.training.common_utils import shard, get_metrics
from flax.training.checkpoints import restore_checkpoint
from flax.training.common_utils import shard, get_metrics
from jax import random

def runner(params, devices, verbose):
    for run_id in range(params["num_runs"]):
        print("Run: ", run_id)

        # set batch size depending on the number of devices
        batch_size = max(len(devices) - 1, 1)

        # get data loaders and ground truth targets
        if params['disease'] == "COVID":
            if params["joint"]:
                (
                    train_loader,
                    metas_train_dim,
                    X_train_dim,
                    seqlen,
                ) = fetch_county_data_covid(
                    params["state"],
                    "all",
                    pred_week=params["pred_week"],
                    batch_size=batch_size,
                    noise_level=params["noise_level"],
                )
            else:
                (
                    train_loader,
                    metas_train_dim,
                    X_train_dim,
                    seqlen,
                ) = fetch_county_data_covid(
                    params["state"],
                    params["county_id"],
                    pred_week=params["pred_week"],
                    batch_size=batch_size,
                    noise_level=params["noise_level"],
                )
            params["num_steps"] = seqlen
        elif params['disease'] == "Flu":
            if params["joint"]:
                (
                    train_loader,
                    metas_train_dim,
                    X_train_dim,
                    seqlen,
                ) = fetch_county_data_flu(
                    params["state"],
                    "all",
                    pred_week=params["pred_week"],
                    batch_size=batch_size,
                    noise_level=params["noise_level"],
                )
            else:
                (
                    train_loader,
                    metas_train_dim,
                    X_train_dim,
                    seqlen,
                ) = fetch_county_data_flu(
                    params["state"],
                    params["county_id"],
                    pred_week=params["pred_week"],
                    batch_size=batch_size,
                    noise_level=params["noise_level"],
                )
            params["num_steps"] = seqlen * 7

        # add days ahead to num steps because num steps is used for forward pass of param model
        training_num_steps = params["num_steps"]
        params["num_steps"] += DAYS_HEAD
        param_model = build_param_model(
            params, metas_train_dim, X_train_dim, devices[0], CUSTOM_INIT=True
        )
        # filename to save/load model
        file_name = "param_model" + "_" + params["model_name"]
        # do not train ABM because it uses a different calibration procedure
        train_flag = (
            False
            if params["model_name"].startswith("ABM") or params["inference_only"]
            else True
        )

        num_epochs = NUM_EPOCHS_DIFF
        CLIP = 10
        if "learnable-params" in params["model_name"]:
            lr = 1e-2  # obtained after tuning
            num_epochs *= 2
        else:
            lr = 1e-4 if params["model_name"].startswith("GradABM") else 1e-4

        """ step 1: training  """
        if train_flag:
            assert param_model != None
            optimizer_def = torch.optim.Adam(
            filter(lambda p: p.requires_grad, param_model.init_parameters()['params'].values()),
            lr=lr,
            weight_decay=0.01)
            optimizer = optimizer_def.create(param_model)
            losses = []
            rng = jax.random.PRNGKey(0)
            for epi in tqdm(range(num_epochs)):
                start = time.time()
                batch_predictions = []
                if verbose:
                    print("\n", "=" * 60)
                    print("Epoch: ", epi)
                epoch_loss = 0
                print(len(train_loader))
                for batch, (counties, meta, x, y) in enumerate(train_loader):
                    print(batch, counties)
                    # construct abm for each forward pass
                    abm = build_simulator(copy(params), devices, counties)
                    # forward pass param model
                    meta = meta.to(devices[0])
                    x = x.to(devices[0])
                    y = y.to(devices[0])
                    param_values = param_model_forward(param_model, params, x, meta)
                    if verbose:
                        if param_values.ndim > 2:
                            print(param_values[:, [0, -1], :])
                        else:
                            print(param_values)
                    # forward simulator for several time steps
                    if BENCHMARK_TRAIN:
                        start_bench = time.time()
                    predictions = forward_simulator(
                        params, param_values, abm, training_num_steps, counties, devices
                    )
                    if BENCHMARK_TRAIN:
                        # quit after 1 epoch
                        print("No steps:", training_num_steps)
                        print("time (s): ", time.time() - start_bench)
                        quit()
                    # loss
                    if verbose:
                        print(jnp.concatenate((y, predictions), 2))
                    loss_weight = jnp.ones((len(counties), training_num_steps, 1)).to(
                        devices[0]
                    )
                    loss = jnp.mean((loss_weight * jnp.square(y - predictions)).mean())
                    grad_fn = jax.value_and_grad(loss_fn)
                    optimizer, grad = grad_fn(optimizer, x, y)
                    optimizer = optimizer.apply_gradient(grad)
                    epoch_loss += jnp.sqrt(loss).item()
                    print("current Loss is - ", epoch_loss)
                losses.append(epoch_loss / (batch + 1))  # divide by number of batches
                print("total Loss is - ", epoch_loss)
                if verbose:
                    print("epoch_loss", epoch_loss)

                """ save best model """
                if epoch_loss < best_loss:
                    if params["joint"]:
                        save_model(
                            param_model,
                            file_name,
                            params['disease'],
                            "joint",
                            params["pred_week"],
                        )
                    else:
                        save_model(
                            param_model,
                            file_name,
                            params['disease'],
                            params["county_id"],
                            params["pred_week"],
                        )
                    best_loss = epoch_loss

                print("epoch {} time (s): {:.2f}".format(epi, time.time() - start))

        """ step 2: inference step  """
        """ upload best model in inference """
        param_model = None
        abm = None
        param_model = build_param_model(
            copy(params), metas_train_dim, X_train_dim, devices[0], CUSTOM_INIT=True
        )
        if not params["model_name"].startswith("ABM"):
            # load param model if it is not ABM-expert
            if params["joint"]:
                param_model = load_model(
                    param_model,
                    file_name,
                    params['disease'],
                    "joint",
                    params["pred_week"],
                    devices[0],
                )
            else:
                param_model = load_model(
                    param_model,
                    file_name,
                    params['disease'],
                    params["county_id"],
                    params["pred_week"],
                    devices[0],
                )
        elif "ABM-pred-correction" in params["model_name"]:
            # pred-correction, uses param model from ODE
            file_name = "param_model" + "_" + "DiffODE-learnable-params"
            if params["noise_level"] > 0:
                file_name = (
                    "param_model"
                    + "_"
                    + "DiffODE-learnable-params"
                    + "-noise"
                    + str(params["noise_level"])
                )
            param_model = load_model(
                param_model,
                file_name,
                params['disease'],
                params["county_id"],
                params["pred_week"],
                devices[0],
            )

        num_step = training_num_steps + DAYS_HEAD
        batch_predictions = []
        counties_predicted = []
        learned_params = []
        with torch.no_grad():
            for batch, (counties, meta, x, y) in enumerate(train_loader):
                # construct abm for each forward pass
                abm = build_simulator(params, devices, counties)
                # forward pass param model
                meta = meta.to(devices[0])
                x = x.to(devices[0])
                param_values = param_model_forward(param_model, params, x, meta)
                # forward simulator for several time steps
                preds = forward_simulator(
                    params, param_values, abm, num_step, counties, devices
                )
                batch_predictions.append(preds)
                counties_predicted.extend(counties)
                learned_params.extend(np.array(param_values.cpu().detach()))
        predictions = torch.cat(batch_predictions, axis=0)
        # we only care about the last predictions
        # predictions are weekly, so we only care about the last 4
        if params['disease'] == "Flu":
            predictions = predictions.squeeze(2)[:, -DAYS_HEAD // 7 :]
        else:
            predictions = predictions.squeeze(2)[:, -DAYS_HEAD:]
        """ remove grad """
        predictions = predictions.cpu().detach()

        """ release memory """
        param_model = None
        abm = None
        torch.cuda.empty_cache()

        """ plot losses """
        # only if trained
        if train_flag:
            disease = params['disease']
            if params["joint"]:
                FIGPATH = f"/kaggle/working/Figures/{disease}/joint/"
            else:
                county_id = params["county_id"]
                FIGPATH = f"/kaggle/working/Figures/{disease}/{county_id}/"
            if not os.path.exists(FIGPATH):
                os.makedirs(FIGPATH)
            sns.set()
            fig = plt.figure()
            ax = fig.add_subplot(1, 1, 1)
            ax.plot(losses)
            pred_week = params["pred_week"]
            fig.savefig(FIGPATH + f"/losses_{pred_week}.png")
        print("-" * 60)
        return counties_predicted, np.array(predictions), learned_params

def train_step(optimizer, batch):
    inputs, targets = batch

    def loss_fn(params):
        # Compute your loss function
        predictions = model.apply(params, inputs)
        loss = your_loss_function(predictions, targets)  # Replace with your actual loss function
        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grad = grad_fn(optimizer.target)
    optimizer = optimizer.apply_gradient(grad)
    metrics = {"loss": loss}

    return optimizer, metrics

def train_predict(args):
    # Setting seed
    print("=" * 60)
    if args.joint:
        print(f"state {args.state} week {args.pred_week}")
    else:
        print(f"county {args.county_id} week {args.pred_week}")
    print("Seed used for python random, numpy, and torch is {}".format(args.seed))
#     random.seed(args.seed)
    np.random.seed(args.seed)
    jax.random.PRNGKey(args.seed)

    params = {}
    params["seed"] = args.seed
    params["num_runs"] = args.num_runs
    params['disease'] = args.disease
    params["pred_week"] = args.pred_week
    params["joint"] = args.joint
    params["inference_only"] = args.inference_only
    params["noise_level"] = args.noise  # for robustness experiments
    # state
    params["state"] = args.state
    if params["joint"]:
        # verify it is a state
        assert params["state"] in states
    else:
        params["county_id"] = args.county_id
        # verify county belongs to state
        assert params["county_id"] in counties[params["state"]]
    params["model_name"] = args.model_name

    devices = jax.devices(backend = None)
#     if args.dev == ["cpu"]:
#         devices = [jax.devices("cpu")[0]]
#     else:
#         devices = [jax.devices(f"cuda:{i}")[0] for i in args.dev]

    print("devices used:", devices)
    verbose = False
    # Get the learned parameters and make predictions
    counties_predicted, predictions, learned_params = runner(params, devices, verbose)

    return counties_predicted, predictions, learned_params

---- MAIN IMPORTS SUCCESSFUL -----


# Runner

In [256]:
def save_predictions(
    disease: str,
    model_name: str,
    region: str,
    pred_week: str,
    death_predictions: np.ndarray,
):
    """
    Given an array w/ predictions, save as csv
    """
    data = np.array([np.arange(len(death_predictions)) + 1, death_predictions])
    if disease == "COVID":
        df = pd.DataFrame(data.transpose(), columns=["k_ahead", "deaths"])
    elif disease == "Flu":
        df = pd.DataFrame(data.transpose(), columns=["k_ahead", "ili"])
    else:
        raise ValueError("disease must be COVID or Flu")
    df["k_ahead"] = df["k_ahead"].astype("int8")
    path = f"./Results/{disease}/{region}/"
    if not os.path.exists(path):
        os.makedirs(path)
    file_name = f"preds_{model_name}_{pred_week}.csv"
    df.to_csv(os.path.join(path, file_name), index=False)


def save_params(
    disease: str,
    model_name: str,
    region: str,
    pred_week: str,
    param_values: np.ndarray,
):
    """
    Given an array w/ predictions, save as csv
    """
    path = f"./Results/{disease}/{region}/"
    if not os.path.exists(path):
        os.makedirs(path)
    file_name = f"params_{model_name}_{pred_week}.csv"
    np.savetxt(os.path.join(path, file_name), param_values, delimiter=",")

In [257]:
parser = argparse.ArgumentParser(description="GradABM for COVID-19 and Flu.")
parser.add_argument("-m", "--model_name", help="Model name.", default="GradABM-time-varying")
parser.add_argument(
    "-di", "--disease", help="Disease: COVID or Flu.", default="COVID"
)
parser.add_argument(
    "-s",
    "--seed",
    type=int,
    help="Seed for python random, numpy and torch",
    default=6666,
)
parser.add_argument("-n", "--num_runs", type=int, help="Number of runs", default=1)
parser.add_argument("-st", "--state", help="State to predict", default="MA")
parser.add_argument(
    "-c",
    "--county_id",
    help="County to predict, only when not using joint training",
    default="25001",
)
parser.add_argument(
    "-d",
    "--dev",
    nargs="+",
    type=str,
    default="0",
    help="Device number to use. Put list for multiple.",
)
parser.add_argument(
    "-ew",
    "--pred_ew",
    type=str,
    default="202021",
    help="Prediction week in CDC format",
)
parser.add_argument(
    "-j", "--joint", action="store_true", help="Train all counties jointly"
)
parser.add_argument(
    "-i",
    "--inference_only",
    action="store_true",
    help="Will not train if True, inference only",
)
parser.add_argument(
    "-no",
    "--noise",
    type=int,
    help="Noise level for robustness experiments",
    default=0,
)
parser.add_argument(
    "-f",
    "--results_file_postfix",
    help="Postfix to be appended to output dir for ease of interpretation",
    default="",
)
parser.set_defaults(joint=True)  # make true when removing no joint
parser.set_defaults(inference_only=False)  # make true when removing no joint
args = parser.parse_args()

# get list of epiweeks for iteration
disease = args.disease
model_name = args.model_name
pred_ew = Week.fromstring(args.pred_ew)

def run_all_weeks(args):
    args.pred_week = pred_ew.cdcformat()
    try:
        counties_predicted, predictions, learned_params = train_predict(args)
        num_counties = len(counties_predicted)
        for c in tqdm(range(num_counties)):
            save_predictions(
                disease,
                model_name,
                counties_predicted[c],
                str(pred_ew),
                predictions[c, :],
            )
            save_params(
                disease,
                model_name,
                counties_predicted[c],
                str(pred_ew),
                learned_params[c],
            )
    except Exception as e:
        print(
            f"exception: did not work for {args.state} week {pred_ew}: "
            + str(e)
            + "\n"
        )
        traceback.print_exc()

In [258]:
run_all_weeks(copy(args))

state MA week 202021
Seed used for python random, numpy, and torch is 6666
devices used: [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
Run:  0
exception: did not work for MA week 202021: "CalibNN" object has no attribute "init_parameters". If "init_parameters" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.



Traceback (most recent call last):
  File "/tmp/ipykernel_31/318974099.py", line 70, in run_all_weeks
    counties_predicted, predictions, learned_params = train_predict(args)
  File "/tmp/ipykernel_31/22302166.py", line 709, in train_predict
    counties_predicted, predictions, learned_params = runner(params, devices, verbose)
  File "/tmp/ipykernel_31/22302166.py", line 474, in runner
    filter(lambda p: p.requires_grad, param_model.init_parameters()['params'].values()),
  File "/opt/conda/lib/python3.10/site-packages/flax/linen/module.py", line 937, in __getattr__
    raise AttributeError(msg)
AttributeError: "CalibNN" object has no attribute "init_parameters". If "init_parameters" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.
